# 코드 샘플로 이해하는 Microsoft Agent Framework

 

----

Microsoft Agent Framework는 프로덕션 수준의 에이전틱 AI 애플리케이션을 만들기 위한 오픈 소스 엔진입니다. Semantic Kernel의 엔터프라이즈급 기반과 AutoGen의 혁신적인 오케스트레이션 패턴을 통합하여, 실험과 프로덕션 배포를 모두 아우르는 단일 프레임워크를 제공합니다.

 

| 구성 요소 | 설명 |
|------------|-------------|
| **1. Chat Agents** | Azure OpenAI, OpenAI, Azure AI 등과 같은 다양한 챗 클라이언트를 사용해 대화형 상호작용을 처리하는 핵심 에이전트 |
| **2. Tools & Functions** | 코드 인터프리터, 파일 검색, Bing 그라운딩과 같은 내장 도구 및 사용자가 확장할 수 있는 커스텀 함수 도구 |
| **3. Multi-Agent Orchestration** | 여러 에이전트를 조율하기 위한 시퀀셜, 병렬, 그룹 채팅, 핸드오프, Magentic 패턴 지원 |
| **4. Open Standards** | 상호 운용성을 위한 MCP(Model Context Protocol), A2A(Agent-to-Agent), OpenAPI를 네이티브로 지원 |

![Microsoft Agent Framework](https://devblogs.microsoft.com/foundry/wp-content/uploads/sites/89/2025/09/AgentStack.png)


## Microsoft Agent Framework 아키텍처

---



Microsoft Agent Framework는 Semantic Kernel과 AutoGen을 통합·발전시킨 형태로, 두 세계의 장점을 하나로 모은 프레임워크입니다.



**핵심 설계 원칙:**



1. **Open Standards First** - MCP, A2A, OpenAPI를 기반으로 최대한의 상호 운용성 제공

2. **연구에서 프로덕션까지** - Microsoft Research의 최신 오케스트레이션 패턴을 프로덕션 환경에서도 바로 활용 가능

3. **확장 가능한 구조** - 모듈식 아키텍처로, 필요한 컴포넌트를 쉽게 교체·추가 가능

4. **엔터프라이즈 준비 완료** - 관측성, 보안, 내구성, Human-in-the-loop 기능을 기본 제공



**Semantic Kernel과 비교:**

- Kernel 인스턴스에 대한 결합 없이 더 단순한 에이전트 생성

- 에이전트에 스레드 관리 기능이 내장됨

- 데코레이터 기반이 아닌 인라인 툴 등록 방식

- Azure AI Foundry Agent Service와 직접적인 통합 지원



**AutoGen과 비교:**

- 역할이 명확한 단일 메시지 타입(ChatMessage)로 통일

- 체크포인트를 포함한 그래프 기반 Workflow API 제공

- OpenTelemetry 기반 관측 기능 내장

- 멀티 에이전트 시스템의 조합성과 내구성이 강화됨



## 더 알아보기

- [Microsoft Agent Framework 공식 문서](https://learn.microsoft.com/en-us/agent-framework/)

- [GitHub 저장소](https://github.com/microsoft/agent-framework)

- [Azure AI Foundry](https://ai.azure.com/)

- [Agent Framework 블로그](https://devblogs.microsoft.com/foundry/introducing-microsoft-agent-framework-the-open-source-engine-for-agentic-ai-apps/)





이 노트북에서는 Azure AI 및 Azure OpenAI를 백엔드 서비스로 사용하여 Microsoft Agent Framework의 핵심 기능을 코드로 살펴봅니다.


# 환경 설정 (Setup)


Microsoft Agent Framework 패키지를 설치합니다. 프레임워크는 모듈형 구조이므로, 필요한 구성 요소만 골라 설치할 수 있습니다.


노트북에서 사용할 필수 라이브러리들을 임포트합니다.


In [1]:
import asyncio
import os
from typing import Annotated
from dotenv import load_dotenv

# Microsoft Agent Framework core
from agent_framework import ChatAgent, ChatMessage, TextContent, Role, ai_function

# Azure integrations
from agent_framework.azure import AzureOpenAIChatClient, AzureAIAgentClient
from azure.identity.aio import DefaultAzureCredential, AzureCliCredential

load_dotenv(override=True)
print("✓ Libraries imported successfully")

✓ Libraries imported successfully


환경 변수를 설정합니다. Azure OpenAI 또는 Azure AI Foundry 엔드포인트 중 하나를 선택해서 사용할 수 있습니다.


In [2]:
# Check required environment variables
print("Checking environment variables...")

# For Azure OpenAI
azure_openai_endpoint = os.getenv("AZURE_OPENAI_ENDPOINT")
azure_openai_key = os.getenv("AZURE_OPENAI_API_KEY")
azure_openai_deployment = os.getenv("AZURE_OPENAI_CHAT_DEPLOYMENT_NAME")
azure_openai_version = os.getenv("AZURE_OPENAI_API_VERSION")

# For Azure AI Foundry
azure_project_endpoint = os.getenv("AZURE_AI_PROJECT_ENDPOINT")
azure_ai_deployment = os.getenv("AZURE_OPENAI_CHAT_DEPLOYMENT_NAME")

if azure_openai_endpoint and azure_openai_deployment:
    print("✓ Azure OpenAI configuration found")
    SERVICE_TYPE = "azure_openai"
else:
    print("❌ Missing required environment variables!")
    print("\nFor Azure OpenAI, set:")
    print("  - AZURE_OPENAI_ENDPOINT")
    print("  - AZURE_OPENAI_API_KEY")
    print("  - AZURE_OPENAI_CHAT_DEPLOYMENT_NAME")
    print("\nOR for Azure AI Foundry, set:")
    print("  - AZURE_AI_PROJECT_ENDPOINT")
    print("  - AZURE_OPENAI_CHAT_DEPLOYMENT_NAME")
    
print(f"\nUsing service: {SERVICE_TYPE}")
azure_ai_deployment

Checking environment variables...
✓ Azure OpenAI configuration found

Using service: azure_openai


'gpt-4.1-mini'

# Case 1: 기본 에이전트 생성 및 실행



Microsoft Agent Framework에서 에이전트를 만드는 과정은 매우 직관적입니다. Kernel 인스턴스를 요구하는 Semantic Kernel과 달리, Agent Framework에서는 챗 클라이언트에서 바로 에이전트를 생성합니다.


## 🧪 Case 1.1: Azure OpenAI Client를 사용하는 간단한 에이전트

---

AzureOpenAIChatClient를 사용해 기본 ChatAgent를 생성합니다. 에이전트는 스레드 관리를 자동으로 처리합니다.


In [3]:
# Create a chat client and agent with Azure OpenAI
azure_openai_chat_client = AzureOpenAIChatClient(
    deployment_name=azure_openai_deployment,
    endpoint=azure_openai_endpoint,
    api_key=azure_openai_key,
    api_version=azure_openai_version
)

# Create an agent - no Kernel needed!
agent = ChatAgent(
    chat_client=azure_openai_chat_client,
    instructions="You are a helpful assistant that provides concise, informative answers.",
    name="SimpleAssistant"
)

# Run the agent with a simple question
async def run_simple_agent():
    result = await agent.run("Tell me about Microsoft in 3 sentences.")
    print(f"Agent: {result.text}")

await run_simple_agent()

Agent: Microsoft is a global technology company founded in 1975 by Bill Gates and Paul Allen, known for developing the Windows operating system and Office productivity suite. It is a leader in cloud computing, software development, and hardware products such as the Surface lineup and Xbox gaming consoles. Microsoft also offers services like Azure cloud platform and LinkedIn, playing a significant role in both consumer and enterprise technology markets.


## 🧪 Case 1.2: 스트리밍 응답

---

Agent Framework는 스트리밍 응답을 지원하므로, 토큰이 생성되는 대로 실시간으로 출력할 수 있습니다.


In [4]:
async def run_streaming_agent():
    print("Agent: ", end="", flush=True)
    async for update in agent.run_stream("Explain what agentic AI is in simple terms."):
        if update.text:
            print(update.text, end="", flush=True)
    print()  # New line after streaming

await run_streaming_agent()

Agent: Agentic AI is a type of artificial intelligence that can act independently toAgentic AI is a type of artificial intelligence that can act independently to achieve goals. It can make achieve goals. It can make decisions, take actions on its own, and adapt decisions, take actions on its own, and adapt to different situations without needing constant human guidance. Think of it to different situations without needing constant human guidance. Think of it like a smart helper that can figure things out and do tasks by like a smart helper that can figure things out and do tasks by itself.
 itself.


## 🧪 Case 1.3: 스레드를 활용한 멀티 턴 대화

---

에이전트는 스레드를 사용해 대화 맥락을 자동으로 유지합니다.


In [5]:
async def multi_turn_conversation():
    
    # Create a new thread.
    thread = agent.get_new_thread()
    
    response1 = await agent.run("My name is Alex and I love Python programming.", thread=thread)
    print(f"Agent: {response1.text}\n")
    
    # Get the thread from the result
    serialized = await thread.serialize()

    # Later, deserialize and continue conversation
    new_thread = await agent.deserialize_thread(serialized)
    
    # Second message - uses the same thread to maintain context
    response2 = await agent.run("What's my name?", thread=new_thread)
    print(f"Agent: {response2.text}\n")
    
    # Third message - agent remembers both previous exchanges
    response3 = await agent.run("What programming language did I say I love?", thread=new_thread)
    print(f"Agent: {response3.text}")

await multi_turn_conversation()

Agent: Nice to meet you, Alex! If you ever want to talk about Python programming or need help with any coding questions, feel free to ask!

Agent: Your name is Alex.

Agent: Your name is Alex.

Agent: You said you love Python programming.
Agent: You said you love Python programming.


# Case 2: 커스텀 도구/함수를 가진 에이전트

---

Agent Framework에서는 `@ai_function` 데코레이터를 사용해 커스텀 함수를 간단히 확장 도구로 등록할 수 있습니다. <br>

별도의 플러그인 래퍼 없이 함수를 정의하고 에이전트에 등록하기만 하면 됩니다.


### 🧪 Case 2.1.1: 커스텀 함수 도구를 사용하는 Azure OpenAI 클라이언트 (Local)


In [6]:
from pydantic import Field

# Define tools as standalone functions (not class methods)
@ai_function(description="Get the specials from the menu")
def get_specials() -> str:
    """Returns the daily specials."""
    print("🔧 get_specials 함수가 호출되었습니다!")
    return """
    Special Soup: Clam Chowder
    Special Salad: Cobb Salad
    Special Drink: Chai Tea
    """

@ai_function(description="Get the price of a menu item")
def get_item_price(menu_item: Annotated[str, Field(description="The name of the menu item")]) -> str:
    """Returns the price of the requested item."""
    print(f"🔧 get_item_price 함수가 호출되었습니다! 메뉴 아이템: {menu_item}")
    return "$9.99"

# Create agent using Azure OpenAI Chat Client (Local execution) - NEW AGENT SYNTAX
async def azure_openai_chat_client_demo():
    
    # For authentication, run `az login` command in terminal
    credential = AzureCliCredential()
    
    # Create chat client with proper configuration
    azure_openai_chat_client = AzureOpenAIChatClient(
        credential=credential,
        endpoint=azure_openai_endpoint,
        deployment_name=azure_openai_deployment
    )
    
    # NEW: Create agent using create_version (new agents API)
    ai_agent = ChatAgent(
        chat_client=azure_openai_chat_client,
        instructions="You are a helpful restaurant assistant. Use tools and answer questions about the menu.",
        name="MenuAssistant(AOAIChatClient)",
        tools=[get_specials, get_item_price],  # Pass functions directly
    )

    # Test the agent
    questions = [
        "What are today's specials?",
        "How much does the Clam Chowder cost?"
    ]
    
    # NEW: Using conversation-based approach instead of thread-based
    for question in questions:
        print(f"\nUser: {question}")
        
        # NEW: Responses API approach - simpler than runs
        result = await ai_agent.run(question)
        print(f"Agent: {result.text}")

await azure_openai_chat_client_demo()


User: What are today's specials?


🔧 get_specials 함수가 호출되었습니다!
Agent: Today's specials are:
- Special Soup: Clam Chowder
- Special Salad: Cobb Salad
- Special Drink: Chai Tea

Would you like to know the prices for any of these specials?

User: How much does the Clam Chowder cost?
Agent: Today's specials are:
- Special Soup: Clam Chowder
- Special Salad: Cobb Salad
- Special Drink: Chai Tea

Would you like to know the prices for any of these specials?

User: How much does the Clam Chowder cost?
🔧 get_item_price 함수가 호출되었습니다! 메뉴 아이템: Clam Chowder
🔧 get_item_price 함수가 호출되었습니다! 메뉴 아이템: Clam Chowder
Agent: The Clam Chowder costs $9.99.
Agent: The Clam Chowder costs $9.99.


### 🧪 Case 2.1.2: 커스텀 함수 도구를 사용하는 Foundry Agent Client (on Cloud)


In [7]:
# Azure CLI 로그인은 터미널에서 먼저 수행해주세요:

#!az login --use-device-code

# !az account set --subscription "your-subscription-id"



# 현재 로그인 상태 및 subscription 확인

!az account show --output table



**************************************************************************************************************
* WARNING:                                                                                                   *
* Extension "azure-cli-ml" cannot be used along with extension "ml". This may result in unexpected behaviour.*
* Please remove azure-cli-ml extension by running  "az extension remove -n azure-cli-ml                      *
**************************************************************************************************************
                
EnvironmentName    HomeTenantId                          IsDefault    Name                                     State    TenantDefaultDomain    TenantDisplayName         TenantId
-----------------  ------------------------------------  -----------  ---------------------------------------  -------  ---------------------  ------------------------  ------------------------------------
AzureCloud         16b3c013-d300-468d-ac64-

In [8]:
# Menu class
class MenuTool:
    #Azure AI Agent doesn't need to use decorator
    #@ai_function(description="Get the specials from the menu")
    def get_specials(self) -> str:
        """Returns the daily specials."""
        return """
        Special Soup: Clam Chowder
        Special Salad: Cobb Salad
        Special Drink: Chai Tea
        """

    #@ai_function(description="Get the price of a menu item")
    def get_item_price(self, menu_item: Annotated[str, "The name of the menu item"]) -> str:
        """Returns the price of the requested item."""
        return "$9.99"

# Create agent using Azure AI
async def azure_ai_agent_demo():

    credential = AzureCliCredential()
    
    # Create an instance of the tool
    menu_tool = MenuTool()
    async with AzureAIAgentClient(
        async_credential=credential,
        project_endpoint=azure_project_endpoint,
        model_deployment_name=azure_ai_deployment
    ) as azure_ai_agent_client:
    
        ai_agent = ChatAgent(
            chat_client=azure_ai_agent_client,
            instructions="You are a helpful restaurant assistant. Answer questions about the menu.",
            name="MenuAgent(FoundryAgent)",
            tool_choice="auto",
            tools=[menu_tool.get_specials, menu_tool.get_item_price],
            # When conversation_id is set, store must be True for service-managed threads.
            store=True
        )

        # Test the agent - moved inside the context manager
        questions = [
            "What are today's specials?",
            "How much does the Clam Chowder cost?"
        ]
        
        for question in questions:
            print(f"\nUser: {question}")
            result = await ai_agent.run(question)
            print(f"Agent: {result.text}")

await azure_ai_agent_demo()


User: What are today's specials?
Agent: Today's specials are:
- Special Soup: Clam Chowder
- Special Salad: Cobb Salad
- Special Drink: Chai Tea

Would you like to know more about any of these specials?

User: How much does the Clam Chowder cost?
Agent: Today's specials are:
- Special Soup: Clam Chowder
- Special Salad: Cobb Salad
- Special Drink: Chai Tea

Would you like to know more about any of these specials?

User: How much does the Clam Chowder cost?
Agent: The Clam Chowder costs $9.99.
Agent: The Clam Chowder costs $9.99.


### 🧪 Case 2.1.3: 커스텀 함수 도구를 사용하는 Azure OpenAI Responses 클라이언트


In [9]:
from agent_framework.azure import AzureOpenAIResponsesClient
from azure.identity import AzureCliCredential

# Menu class
class MenuTool:
    def get_specials(self) -> str:
        """Returns the daily specials."""
        return """
        Special Soup: Clam Chowder
        Special Salad: Cobb Salad
        Special Drink: Chai Tea
        """

    def get_item_price(self, menu_item: Annotated[str, "The name of the menu item"]) -> str:
        """Returns the price of the requested item."""
        return "$9.99"

# Create agent using Azure OpenAI Responses Client
async def azure_openai_responses_demo():
    """
    Demonstrates AzureOpenAIResponsesClient with both non-streaming and streaming responses.
    This client provides structured response generation using Azure OpenAI.
    """
    
    # For authentication, run `az login` command in terminal
    menu_tool = MenuTool()
    
    # Create agent using AzureOpenAIResponsesClient
    agent = AzureOpenAIResponsesClient(
            endpoint=azure_openai_endpoint,
            api_key=azure_openai_key,
            deployment_name=azure_openai_deployment,
            api_version="preview" # Use "preview" for Azure OpenAI Responses. Currently, the api_version must be "preview".
            ).create_agent(
        instructions="You are a helpful restaurant assistant. Answer questions about the menu.",
        tools=[menu_tool.get_specials, menu_tool.get_item_price],
    )
    
    # Test questions
    questions = [
        "What are today's specials?",
        "How much does the Clam Chowder cost?"
    ]
    
    # 1. Non-streaming Example (get complete result at once)
    print("=== Non-streaming Response ===\n")
    for question in questions:
        print(f"User: {question}")
        result = await agent.run(question)
        print(f"Agent: {result.text}\n")
    
    print("-" * 80 + "\n")
    
    # 2. Streaming Example (get results as they are generated)
    print("=== Streaming Response ===\n")
    for question in questions:
        print(f"User: {question}")
        print("Agent: ", end="", flush=True)
        async for chunk in agent.run_stream(question):
            if chunk.text:
                print(chunk.text, end="", flush=True)
        print("\n")

await azure_openai_responses_demo()

=== Non-streaming Response ===

User: What are today's specials?


Agent: Today's specials are:
- Special Soup: Clam Chowder
- Special Salad: Cobb Salad
- Special Drink: Chai Tea

Would you like to know the prices of any of these items or more details?

User: How much does the Clam Chowder cost?
Agent: The Clam Chowder costs $9.99. Would you like to know about any other items?

--------------------------------------------------------------------------------

=== Streaming Response ===

User: What are today's specials?
Agent: Agent: The Clam Chowder costs $9.99. Would you like to know about any other items?

--------------------------------------------------------------------------------

=== Streaming Response ===

User: What are today's specials?
Agent: Today'sToday's specials specials are:
 are:
- Special Soup- Special Soup:: Clam Clam Chowder Chowder

- Special- Special Salad: Salad: Cobb Salad Cobb Salad

- Special- Special Drink: Drink: Chai Chai Tea

 Tea

Would you likeWould you like to to know the know the prices or prices or more more details

# Case 3: 에이전트 클라이언트 간 차이 이해하기



아래 표를 통해 Microsoft Agent Framework의 세 가지 주요 에이전트 클라이언트(`AzureOpenAIChatClient`, `AzureAIAgentClient`, `AzureOpenAIResponsesClient`) 간의 차이를 살펴봅니다.



| Case | 클라이언트 타입 | 사용 도구 | 실행 위치 |
|------|----------------|-----------|-----------|
| **3.1** | `AzureOpenAIChatClient` | 커스텀 `@ai_function` | **로컬(subprocess)** |
| **3.2** | `AzureAIAgentClient` | `HostedCodeInterpreterTool` | **Azure AI 서비스** |
| **3.3** | `AzureOpenAIResponsesClient` | `HostedCodeInterpreterTool` | **Azure OpenAI 서비스** | 



### 언제 어떤 방식을 사용할까요?



- **Case 3.1**를 사용할 때:

  - 코드 실행 환경을 직접 완전히 제어해야 할 때

  - 로컬 환경에서 코드를 실행하고 싶을 때

  - Azure AI 서비스를 사용할 수 없는 환경일 때

  - 특정 파이썬 패키지 등, 맞춤 실행 환경이 필요할 때



- **Case 3.2**를 사용할 때:

  - 엔터프라이즈 애플리케이션을 구축할 때

  - 관리형·보안성이 높은 코드 실행 환경이 필요할 때

  - Azure AI Foundry 기능을 적극적으로 활용하고 싶을 때

  - 다른 Azure AI 서비스와 긴밀하게 통합해야 할 때



- **Case 3.3**를 사용할 때:

  - Azure OpenAI Responses API를 사용할 때

  - 생성된 코드를 추출·검토하고 싶을 때

  - 구조화된 응답 포맷이 필요한 워크로드일 때


## 🧪 Case 3.1: 함수 호출을 사용하는 Azure OpenAI 클라이언트 (subprocess에서 실행)


In [10]:
import subprocess
import tempfile
import os

# Custom Python code execution function
@ai_function(description="Execute Python code and return the result")
def execute_python_code(code: Annotated[str, "Python code to execute"]) -> str:
    """
    Executes Python code in a temporary file and returns the output.
    
    Args:
        code: Python code to execute
        
    Returns:
        The output of the code execution or error message
    """
    try:
        # Create a temporary file to store the code
        with tempfile.NamedTemporaryFile(mode='w', suffix='.py', delete=False) as temp_file:
            temp_file.write(code)
            temp_file_path = temp_file.name
        
        # Execute the Python code
        result = subprocess.run(
            ['python', temp_file_path],
            capture_output=True,
            text=True,
            timeout=5  # 5 seconds timeout for safety
        )
        
        # Clean up
        os.unlink(temp_file_path)
        
        # Return output or error
        if result.returncode == 0:
            return f"Success:\n{result.stdout}" if result.stdout else "Code executed successfully with no output"
        else:
            return f"Error:\n{result.stderr}"
            
    except subprocess.TimeoutExpired:
        os.unlink(temp_file_path)
        return "Error: Code execution timed out (5 seconds limit)"
    except Exception as e:
        return f"Error: {str(e)}"
    
async def code_interpreter_demo_using_subprocess():
    
    thread = agent.get_new_thread()

    code_agent = ChatAgent(
        chat_client=azure_openai_chat_client,
        instructions="You are a helpful assistant that can write and get the result of the Python code.",
        name="CodeAssistant(AOAIChatClient)",
        tools=[execute_python_code]
    )
    
    # Ask the agent to solve a problem using code
    task = "Calculate the first 10 numbers in the Fibonacci sequence. Provide the result as a Python list."
    print(f"User: {task}")
    
    result = await code_agent.run(task, thread=thread)
    print(f"\nAgent: {result.text}")

await code_interpreter_demo_using_subprocess()

User: Calculate the first 10 numbers in the Fibonacci sequence. Provide the result as a Python list.

Agent: The first 10 numbers in the Fibonacci sequence are: [0, 1, 1, 2, 3, 5, 8, 13, 21, 34]

Agent: The first 10 numbers in the Fibonacci sequence are: [0, 1, 1, 2, 3, 5, 8, 13, 21, 34]


## 🧪 Case 3.2: Hosted Code Interpreter Tool을 사용하는 Azure AI Agent 클라이언트 (서버 측 실행)


In [11]:

from agent_framework import HostedCodeInterpreterTool

async def code_interpreter_demo_using_subprocess():
    credential = AzureCliCredential()
    # Create agent with code interpreter capability
    code_agent = ChatAgent(
        chat_client=AzureAIAgentClient(
            async_credential=credential,
            project_endpoint=azure_project_endpoint,
            model_deployment_name=azure_ai_deployment
        ),
        instructions="You are a helpful assistant that can write and execute Python code.",
        name="CodeAssistant(AzureAIAgentClient)",
        tools=[HostedCodeInterpreterTool()]
    )
    
    # Ask the agent to solve a problem using code
    task = "Calculate the first 10 numbers in the Fibonacci sequence."
    print(f"User: {task}")
    
    result = await code_agent.run(task)
    print(f"\nAgent: {result.text}")

await code_interpreter_demo_using_subprocess()


User: Calculate the first 10 numbers in the Fibonacci sequence.



Agent: The Fibonacci sequence is a series of numbers where each number is the sum of the two preceding ones, usually starting with 0 and 1. I will calculate the first 10 numbers in the Fibonacci sequence for you.The first 10 numbers in the Fibonacci sequence are: 0, 1, 1, 2, 3, 5, 8, 13, 21, 34.


## 🧪 Case 3.3: Hosted Code Interpreter Tool을 사용하는 Azure OpenAI Responses 클라이언트 (서버 측 실행)


In [12]:
from agent_framework import ChatAgent, ChatResponse, HostedCodeInterpreterTool
from agent_framework.azure import AzureOpenAIResponsesClient
from azure.identity import AzureCliCredential
from openai.types.responses.response import Response as OpenAIResponse
from openai.types.responses.response_code_interpreter_tool_call import ResponseCodeInterpreterToolCall

async def code_interpreter_responses_demo():
    """
    Demonstrates using HostedCodeInterpreterTool with Azure OpenAI Responses Client.
    
    This example shows how to execute Python code on Azure OpenAI service using
    the Responses API with the hosted code interpreter tool.
    """
    print("=== Azure OpenAI Responses Client with Hosted Code Interpreter ===\n")
    
    # For authentication, run `az login` command in terminal
    agent = ChatAgent(
        chat_client=AzureOpenAIResponsesClient(
            credential=AzureCliCredential(),
            endpoint=azure_openai_endpoint,
            deployment_name=azure_openai_deployment,
            api_version="preview" # Use "preview" for Azure OpenAI Responses. Currently, the api_version must be "preview".
        ),
        instructions="You are a helpful assistant that can write and execute Python code to solve problems.",
        tools=[HostedCodeInterpreterTool()],
    )
    
    # Test questions that require code execution
    questions = [
        "Use code to calculate the factorial of 100",
        "Calculate the first 15 numbers in the Fibonacci sequence",
        "Generate a list of prime numbers less than 100"
    ]
    
    for question in questions:
        print(f"User: {question}")
        result = await agent.run(question)
        print(f"Agent: {result.text}\n")
        
        # Extract and display the generated code if available
        if (
            isinstance(result.raw_representation, ChatResponse)
            and isinstance(result.raw_representation.raw_representation, OpenAIResponse)
            and len(result.raw_representation.raw_representation.output) > 0
            and isinstance(
                result.raw_representation.raw_representation.output[0], 
                ResponseCodeInterpreterToolCall
            )
        ):
            generated_code = result.raw_representation.raw_representation.output[0].code
            print(f"📝 Generated code:\n```python\n{generated_code}\n```\n")
        
        print("-" * 80 + "\n")

await code_interpreter_responses_demo()

=== Azure OpenAI Responses Client with Hosted Code Interpreter ===

User: Use code to calculate the factorial of 100


Agent: import math

factorial_100 = math.factorial(100)
factorial_100 The factorial of 100 is:

93326215443944152681699238856266700490715968264381621468592963895217599993229915608941463976156518286253697920827223758251185210916864000000000000000000000000

📝 Generated code:
```python
import math

factorial_100 = math.factorial(100)
factorial_100
```

--------------------------------------------------------------------------------

User: Calculate the first 15 numbers in the Fibonacci sequence


[2025-12-03 11:37:08 - /anaconda/envs/venv_agentlab/lib/python3.12/asyncio/base_events.py:1833 - ERROR] Unclosed client session
client_session: <aiohttp.client.ClientSession object at 0x704801dd9fd0>
[2025-12-03 11:37:08 - /anaconda/envs/venv_agentlab/lib/python3.12/asyncio/base_events.py:1833 - ERROR] Unclosed connector
connections: ['deque([(<aiohttp.client_proto.ResponseHandler object at 0x704801e938f0>, 6693856.700391032)])']
connector: <aiohttp.connector.TCPConnector object at 0x7048028eb6e0>
[2025-12-03 11:37:08 - /anaconda/envs/venv_agentlab/lib/python3.12/asyncio/base_events.py:1833 - ERROR] Unclosed connector
connections: ['deque([(<aiohttp.client_proto.ResponseHandler object at 0x704801e938f0>, 6693856.700391032)])']
connector: <aiohttp.connector.TCPConnector object at 0x7048028eb6e0>


Agent: The Fibonacci sequence is calculated by starting with 0 and 1, and each subsequent number is the sum of the two preceding ones.

Let's calculate the first 15 numbers in the Fibonacci sequence. def fibonacci_sequence(n):
    sequence = [0, 1]
    for i in range(2, n):
        next_number = sequence[i-1] + sequence[i-2]
        sequence.append(next_number)
    return sequence

# Calculate the first 15 numbers in the Fibonacci sequence
fibonacci_15 = fibonacci_sequence(15)
fibonacci_15 The first 15 numbers in the Fibonacci sequence are:

0, 1, 1, 2, 3, 5, 8, 13, 21, 34, 55, 89, 144, 233, 377

--------------------------------------------------------------------------------

User: Generate a list of prime numbers less than 100
Agent: I will generate a list of prime numbers less than 100 for you. def is_prime(n):
    if n <= 1:
        return False
    if n <= 3:
        return True
    if n % 2 == 0 or n % 3 == 0:
        return False
    i = 5
    while i * i <= n:
        if n % i 

## 🧪 Case 4: Azure AI Agent Client로 살펴보는 엔터프라이즈 도구 활용 예시

---

이 섹션에서는 `agent_framework`의 Azure AI Agent Client와 함께 다양한 방식으로 에이전트와 도구를 생성·사용하는 예시를 다룹니다.


# Case 4: Azure AI Agent Client와 함께하는 엔터프라이즈 도구

---

Microsoft Agent Framework는 Azure 서비스와 긴밀하게 통합되는 강력한 엔터프라이즈급 호스티드 도구들을 제공합니다. 이 도구들을 활용하면, 웹 검색·문서 검색·외부 시스템 연동 등 고급 기능을 에이전트에 쉽게 추가할 수 있습니다.



## 호스티드 도구 비교



| 도구 | 목적 | 서비스 | 주요 특징 |
|------|---------|---------|-----------|
| **HostedFileSearchTool** | 문서/데이터 검색 | Azure AI Search | ✅ 인덱스 기반 검색<br>✅ 시맨틱 검색<br>✅ 랭킹 튜닝 가능 |
| **HostedWebSearchTool** | 실시간 웹 검색 | Bing Grounding | ✅ 최신 정보<br>✅ 출처 인용<br>✅ 웹 기반 그라운딩 |
| **Local MCP** | 커스텀 통합 | 로컬 서버 | ✅ 완전한 제어<br>✅ 맞춤 비즈니스 로직<br>✅ 로컬 리소스 활용 |
| **Hosted MCP** | 외부/엔터프라이즈 서비스 | 원격 서버 | ✅ 서드파티 API 연동<br>✅ 확장 가능한 인프라<br>✅ 관리형 운영 환경 |



### Case 4를 위한 사전 준비



**Azure AI Search (Case 4.1)용:**

- 데이터가 인덱싱된 Azure AI Search 서비스

- Azure AI Project 내에서 구성된 Search 연결

- 인덱스 이름(예: `"hotels-sample-index"`)



**Bing Grounding (Case 4.2)용:**

- Azure AI Project에 설정된 Bing Grounding 연결

- `BING_CONNECTION_NAME` 또는 `BING_CONNECTION_ID` 환경 변수



**MCP 도구 (Case 4.3 & 4.4)용:**

- MCP(Model Context Protocol)에 대한 기본 이해

- 로컬 또는 호스티드 MCP 서버 설정 정보


## 🧪 Case 4.1: HostedFileSearchTool과 Azure AI Search

---

Azure AI Search에 인덱싱된 호텔 데이터를 활용해 질문에 답하는 예제를 보여줍니다.


> **참고**: 이 케이스를 실행하기 전에 `0_basic-rag` 폴더의 `1_basic-rag.ipynb` 노트북을 먼저 실행해 Azure AI Search 인덱스와 벡터 스토어를 생성해야 합니다.



**사전 준비:**

1. `AZURE_AI_PROJECT_ENDPOINT`, `AZURE_AI_MODEL_DEPLOYMENT_NAME` 환경 변수를 설정합니다.

2. Azure AI 프로젝트 안에 Azure AI Search 연결이 구성되어 있어야 합니다.

3. Azure AI Search 서비스 안에 `"hotels-sample-index"` 인덱스가 존재해야 합니다.

   (이는 **0_basic-rag 폴더**의 **1_basic-rag.ipynb**에서 제공되는 샘플 호텔 데이터를 이용해 생성할 수 있습니다.)



**환경 변수:**

- `AZURE_AI_PROJECT_ENDPOINT`: Azure AI 프로젝트 엔드포인트

- `AZURE_AI_MODEL_DEPLOYMENT_NAME`: 모델 배포 이름



**주요 모듈:**

- `agent_framework`의 `HostedFileSearchTool`

- Azure AI Search 벡터 스토어를 사용한 시맨틱 검색 기능


In [13]:
import os
from azure.identity.aio import AzureCliCredential
from agent_framework.azure import AzureAIAgentClient
from agent_framework import ChatAgent, HostedFileSearchTool, HostedVectorStoreContent
from azure.ai.agents.models import FileInfo, VectorStore

async def employee_search_demo():
    """
    Demonstrates Azure AI agent with file search capabilities using uploaded PDF.
    This example uploads an employee PDF file and creates a vector store for searching.
    """
    
    
    # Create Azure AI Agent client with credential
    credential = AzureCliCredential()
    
    file: FileInfo | None = None
    vector_store: VectorStore | None = None
    
    try:
        # Create a chat client and agent with Azure OpenAI
        azure_openai_chat_client = AzureOpenAIChatClient(
            deployment_name=azure_openai_deployment,
            endpoint=azure_openai_endpoint,
            api_key=azure_openai_key,
            api_version=azure_openai_version
        )

        azure_ai_agent_client = AzureAIAgentClient(
            async_credential=credential,
            project_endpoint=azure_project_endpoint,
            model_deployment_name=azure_ai_deployment
        )
            
        # 1. Upload employee PDF file
        pdf_file_path = "employees.pdf"
        print(f"Uploading file from: {pdf_file_path}")
        
        file = await azure_ai_agent_client.project_client.agents.files.upload_and_poll(
            file_path=str(pdf_file_path), 
            purpose="assistants"
        )
        print(f"Uploaded file, file ID: {file.id}")
        
        # 2. Create vector store with the uploaded file
        vector_store = await azure_ai_agent_client.project_client.agents.vector_stores.create_and_poll(
            file_ids=[file.id], 
            name="employee_vectorstore"
        )
        print(f"Created vector store, ID: {vector_store.id}\n")
        
        # 3. Create file search tool with the vector store
        file_search_tool = HostedFileSearchTool(
            inputs=[HostedVectorStoreContent(vector_store_id=vector_store.id)]
        )
        
        # 4. Create agent with file search capability
        agent = ChatAgent(
            #chat_client=azure_ai_agent_client,
            chat_client=azure_ai_agent_client,
            name="Employee Search Assistant(AzureAIAgentClient)",
            instructions="You are a helpful assistant that can search through uploaded employee files to answer questions about employees.",
            tools=file_search_tool
        )
        
        # 5. Test questions about employees
        USER_INPUTS = [
            "Who is the youngest employee?",
            "Who works in sales?",
            "I have a customer request, who can help me?"
        ]
        
        print("=== Azure AI Agent with File Search (Employee PDF) ===")
        print("This agent can search through employee data to answer questions.\n")
        
        # 6. Simulate conversation with the agent
        for user_input in USER_INPUTS:
            print(f"User: {user_input}")
            print("Agent: ", end="", flush=True)
            
            # Stream the response for better user experience
            async for chunk in agent.run_stream(user_input):
                if chunk.text:
                    print(chunk.text, end="", flush=True)
            print("\n" + "=" * 50 + "\n")
        
        print("Employee search conversation completed!")
            
    finally:
        # 7. Cleanup: Delete the vector store and file
        # Refresh client if needed (agent closes it when used as context manager)
        try:
            cleanup_client = AzureAIAgentClient(
                async_credential=AzureCliCredential(),
                project_endpoint=azure_project_endpoint,
                model_deployment_name=azure_ai_deployment
            )
            await cleanup_client.close()

            if azure_ai_agent_client:
                await azure_ai_agent_client.close()

            if vector_store:
                await cleanup_client.project_client.agents.vector_stores.delete(vector_store.id)
                print(f"Deleted vector store: {vector_store.id}")
            if file:
                await cleanup_client.project_client.agents.files.delete(file.id)
                print(f"Deleted file: {file.id}")
                
    
        except Exception as e:
            print(f"Cleanup warning: {e}")

await employee_search_demo()

Uploading file from: employees.pdf


Uploaded file, file ID: assistant-C39rrmQbWZDWBAZYKN4oY8
Created vector store, ID: vs_RU265lfMmIfNijjTsp4A11qa

=== Azure AI Agent with File Search (Employee PDF) ===
This agent can search through employee data to answer questions.

User: Who is the youngest employee?
Agent: Created vector store, ID: vs_RU265lfMmIfNijjTsp4A11qa

=== Azure AI Agent with File Search (Employee PDF) ===
This agent can search through employee data to answer questions.

User: Who is the youngest employee?
Agent: The youngest employee is Alice Johnson, who isThe youngest employee is Alice Johnson, who is 24 24 years old years old【4:0†employees.pdf】【4:0†employees.pdf】..

User: Who works in sales?
Agent: 

User: Who works in sales?
Agent: Alice Johnson works inAlice Johnson works in the Sales department. 

If you need information on anyone the Sales department. 

If you need information on anyone else else or more or more details details, feel free to ask!【4:0†employees.pdf】, feel free to ask!【4:0†employees.pdf

In [14]:
import os
from agent_framework.azure import AzureAIAgentClient
from agent_framework import HostedFileSearchTool
from agent_framework import ChatAgent


async def ai_search_tool_responses_demo():

    try:
        # Create Azure AI Agent client with credential
        credential = AzureCliCredential()

        
        # Create file search tool with vector store
        azure_ai_search_tool = HostedFileSearchTool(
            additional_properties={
                "index_name": "hotels-sample-index",  # Name of your search index
                "query_type": "simple",  # Use simple search
                "top_k": 10,  # Get more comprehensive results
            },
        )

        azure_openai_chat_client = AzureOpenAIChatClient(
                deployment_name=azure_openai_deployment,
                endpoint=azure_openai_endpoint,
                api_key=azure_openai_key,
                api_version=azure_openai_version
            )

        # TODO: Not work with Azure AI Agent Client yet - https://github.com/microsoft/agent-framework/issues/1381
        # azure_ai_agent_client = AzureAIAgentClient(
        #     async_credential=credential,
        #     project_endpoint=project_endpoint,
        #     model_deployment_name=azure_ai_deployment
        # )
    
        # Create agent with file search capability
        agent = ChatAgent(
            chat_client=azure_openai_chat_client,
            name="Hotel Search Assistant(AzureOpenAIChatClient)",
            instructions="You are a helpful travel assistant that searches hotel information.",
            tools=azure_ai_search_tool
        )

        USER_INPUTS = [
            "Search hotels in USA and give me detailed information.",
        ]
        
        print("=== Azure AI Agent with Azure AI Search ===")
        print("This agent can search through hotel data to help you find accommodations.\n")

        # 3. Simulate conversation with the agent
        for user_input in USER_INPUTS:
            print(f"User: {user_input}")
            print("Agent: ", end="", flush=True)

            # Stream the response for better user experience
            async for chunk in agent.run_stream(user_input):
                if chunk.text:
                    print(chunk.text, end="", flush=True)
            print("\n" + "=" * 50 + "\n")

        print("Hotel search conversation completed!")

    except Exception as e:
        print(f"Error occurred: {e}")
    finally:
        cleanup_client = AzureAIAgentClient(
            async_credential=AzureCliCredential(),
            project_endpoint=azure_project_endpoint,
            model_deployment_name=azure_ai_deployment
        )
        await cleanup_client.close()

await ai_search_tool_responses_demo()

=== Azure AI Agent with Azure AI Search ===
This agent can search through hotel data to help you find accommodations.

User: Search hotels in USA and give me detailed information.
Agent: To provideTo provide

[2025-12-03 11:37:42 - /anaconda/envs/venv_agentlab/lib/python3.12/asyncio/base_events.py:1833 - ERROR] Unclosed client session
client_session: <aiohttp.client.ClientSession object at 0x7048013d6d80>
[2025-12-03 11:37:42 - /anaconda/envs/venv_agentlab/lib/python3.12/asyncio/base_events.py:1833 - ERROR] Unclosed connector
connections: ['deque([(<aiohttp.client_proto.ResponseHandler object at 0x70480143d970>, 6693903.754086271)])']
connector: <aiohttp.connector.TCPConnector object at 0x704801445c10>
[2025-12-03 11:37:42 - /anaconda/envs/venv_agentlab/lib/python3.12/asyncio/base_events.py:1833 - ERROR] Unclosed connector
connections: ['deque([(<aiohttp.client_proto.ResponseHandler object at 0x70480143d970>, 6693903.754086271)])']
connector: <aiohttp.connector.TCPConnector object at 0x704801445c10>


 detailed information about hotels in the USA, could information about hotels in the USA, could you please specify the city or region you're interested in, as well you please specify the city or region you're interested in, as well as any preferences such as any preferences such as budget, hotel type (e.g., luxury, budget, as budget, hotel type (e.g., luxury, budget, boutique), or amenities? boutique), or amenities? This will help me find the most relevant options for you.

Hotel search conversation completed!
 This will help me find the most relevant options for you.

Hotel search conversation completed!


## 🧪 Case 4.2: HostedWebSearchTool과 Bing Grounding

---

Bing Search API를 활용해 웹 그라운딩을 수행하고, 실시간 웹 데이터를 기반으로 질문에 답하는 예제입니다.


**설정:**

- Bing Search 연결이 구성된 Azure AI Project

- 환경 변수: `AZURE_AI_PROJECT_ENDPOINT`, `BING_CONNECTION_NAME`



**주요 모듈:**

- `agent_framework.azure_ai.tools`의 `HostedWebSearchTool`

- 웹 검색 결과와 인용 정보를 포함한 그라운디드(grounded) 응답 제공


In [15]:
import os
from azure.identity.aio import AzureCliCredential
from agent_framework.azure import AzureAIAgentClient, AzureOpenAIChatClient
from agent_framework import HostedWebSearchTool
from agent_framework import ChatAgent


async def bing_grounding_demo():
    """
    Demonstrates using Bing Search API for web grounding to answer questions with real-time web data.
    """
    
    try:
        # Get Azure AI Project endpoint
        bing_connection_id = os.environ.get("BING_GROUNDING_CONNECTION_ID")
        
        # Create Azure AI Agent client with credential
        credential = AzureCliCredential()
        
        # Create web search tool with Bing connection
        web_search_tool = HostedWebSearchTool(
            
            additional_properties={
                "connection_id": bing_connection_id,
                "top_k": 3,  # Get top 3 search results
            }
        )
        
        # TODO: Not work with Azure OpenAI Chat Client
        # azure_openai_chat_client = AzureOpenAIChatClient(
        #     deployment_name=azure_openai_deployment,
        #     endpoint=azure_openai_endpoint,
        #     api_key=azure_openai_key,
        #     api_version=azure_openai_version
        # )
        
        azure_ai_agent_client = AzureAIAgentClient(
            async_credential=credential,
            project_endpoint=azure_project_endpoint,
            model_deployment_name=azure_ai_deployment
        )
        
        # Create agent with web search capability
        agent = ChatAgent(
            #chat_client=azure_openai_chat_client,
            chat_client=azure_ai_agent_client,
            name="Web Research Assistant(AzureAIAgentClient)",
            instructions="You are a helpful research assistant. Use web search to find current information and provide grounded answers with citations.",
            tools=[web_search_tool]
        )
        
        # Test questions that require web search
        USER_INPUTS = [
            "Tell me what day it is today and What is the latest exciting news in South Korea?"
        ]
        
        print("=== Azure AI Agent with Bing Grounding Search ===")
        print("This agent can search the web for current information and provide grounded answers.\n")
        
        # Simulate conversation with the agent
        for user_input in USER_INPUTS:
            print(f"User: {user_input}")
            print("Agent: ", end="", flush=True)
            
            # Stream the response for better user experience
            async for chunk in agent.run_stream(user_input):
                if chunk.text:
                    print(chunk.text, end="", flush=True)
            print("\n" + "=" * 50 + "\n")
        
        print("Web search conversation completed!")
    
    except Exception as e:
        print(f"Error occurred: {e}")
    finally:
        # Cleanup: Close Azure AI Agent client if it was created
        try:
            await azure_ai_agent_client.close()    
        except Exception as cleanup_error:
            print(f"Cleanup warning: {cleanup_error}")

await bing_grounding_demo()

=== Azure AI Agent with Bing Grounding Search ===
This agent can search the web for current information and provide grounded answers.

User: Tell me what day it is today and What is the latest exciting news in South Korea?
Agent: 

[2025-12-03 11:37:47 - /anaconda/envs/venv_agentlab/lib/python3.12/site-packages/agent_framework_azure_ai/_chat_client.py:677 - ERROR] Error processing stream: Error: invalid_tool_input; The specified connection ID '/subscriptions/3d4d3dd0-79d4-40cf-a94e-b4154812c6ca/resourcegroups/rg-hyokeunchoi-7652/providers/microsoft.cognitiveservices/accounts/hyo-ai-foundry-pjt1-resource/projects/hyo-ai-foundry-pjt1/connections/groundingbingsearch' was not found in the project or account connections. Please verify that the connection id in tool input is correct and exists in the project or account.


Error occurred: Error: invalid_tool_input; The specified connection ID '/subscriptions/3d4d3dd0-79d4-40cf-a94e-b4154812c6ca/resourcegroups/rg-hyokeunchoi-7652/providers/microsoft.cognitiveservices/accounts/hyo-ai-foundry-pjt1-resource/projects/hyo-ai-foundry-pjt1/connections/groundingbingsearch' was not found in the project or account connections. Please verify that the connection id in tool input is correct and exists in the project or account.


## 🧪 Case 4.3: 로컬 MCP 서버 연동

---

로컬에서 실행 중인 Model Context Protocol(MCP) 서버에 연결해, 커스텀 도구로 에이전트 기능을 확장하는 방법을 보여줍니다.


**설정:**

- 로컬 MCP 서버 실행 (예: 파일 시스템 도구, 데이터베이스 커넥터 등)

- MCP 서버 URL 또는 설정 정보

- 환경 변수: `AZURE_OPENAI_ENDPOINT`



**주요 모듈:**

- MCP 서버에서 도구를 자동 탐색·등록

- MCP 규격을 준수하는 모든 서버 구현과 함께 동작


In [16]:
import os
from azure.identity.aio import AzureCliCredential
from agent_framework.azure import AzureOpenAIChatClient, AzureAIAgentClient
from agent_framework import ChatAgent, MCPStreamableHTTPTool


async def local_mcp_demo():
    """
    Demonstrates connecting to a local Model Context Protocol (MCP) server 
    to extend agent capabilities with custom tools.
    
    This example shows how to use Microsoft Learn MCP server with Azure AI Agent.
    """
    
    try:
        # Create Azure AI Agent client with credential
        credential = AzureCliCredential()
        
        print("=== Azure AI Agent with Local MCP Server ===")
        print("This agent can use MCP tools to access Microsoft documentation.\n")
        
        # Connect to local MCP server and create agent with MCP tools
        async with (
            AzureAIAgentClient(
                async_credential=credential,
                project_endpoint=azure_project_endpoint,
                model_deployment_name=azure_ai_deployment
            ) as azure_ai_agent_client,
            MCPStreamableHTTPTool(
                name="Local Microsoft Learn MCP",
                url="https://learn.microsoft.com/api/mcp",
            ) as mcp_server
        ):
            # Create agent with MCP tools
            agent = ChatAgent(
                chat_client=azure_ai_agent_client,
                name="Local MCP Assistant(AzureAIAgentClient)",
                instructions="You are a helpful assistant that can help with Microsoft documentation questions using MCP tools.",
                tools=mcp_server,
                store=True
            )
            
            # Test questions that use MCP tools
            USER_INPUTS = [
                "How to create an Azure storage account using az cli? response with a simple answer and an example.",
                "What is Microsoft Agent Framework? response with key concepts and a main python code example."
            ]
            
            # Simulate conversation with the agent
            for user_input in USER_INPUTS:
                print(f"User: {user_input}")
                print("Agent: ", end="", flush=True)
                
                # Stream the response for better user experience
                async for chunk in agent.run_stream(user_input):
                    if chunk.text:
                        print(chunk.text, end="", flush=True)
                print("\n" + "=" * 50 + "\n")
            
            print("MCP tool conversation completed!")
    
    except Exception as e:
        print(f"Error occurred: {e}")
        import traceback
        traceback.print_exc()

await local_mcp_demo()

=== Azure AI Agent with Local MCP Server ===
This agent can use MCP tools to access Microsoft documentation.



User: How to create an Azure storage account using az cli? response with a simple answer and an example.
Agent: To createTo create an Azure storage an Azure storage account account using the Azure CLI using the Azure CLI ( (az cliaz cli), you), you can use can use the command ` the command `az storageaz storage account account create create`.

Example:

```bash`.

Example:

```bash
az
az storage storage account create account create --name --name myst mystorageaccountorageaccount --resource --resource-group myResourceGroup --location east-group myResourceGroup --location eastus --us --skusku Standard_L Standard_LRSRS
``
```

`

ThisThis command creates a command creates a storage account storage account named named "m "mystystorageaccountorageaccount" in the resource group "myResourceGroup" in the resource group "myResourceGroup"" located in located in "east "eastus"us" with with the Standard the Standard_L_LRS (RS (LocallyLocally-re-redunddundant storageant storage)) SKU SKU..

User: 

## 🧪 Case 4.4: Human-in-the-loop를 지원하는 호스티드 MCP 서버 (Azure AI Project)

---

Azure AI Project에 호스팅된 MCP 서버를 사용하여, Human-in-the-loop(사람 검증/승인) 기능을 포함한 워크플로를 구현하는 예제입니다.


**설정:**

- Azure AI Project 내에 호스팅된 MCP 서버 연결 구성

- MCP 서버가 Azure AI Project에 배포 및 등록 완료

- 환경 변수: `AZURE_AI_PROJECT_ENDPOINT`



**주요 모듈:**

- `agent_framework`의 `HostedMCPTool`

- Azure AI 인프라에 호스팅된 MCP 서버에 연결

- 검토·승인 워크플로를 위한 Human-in-the-loop 지원

- 로컬 서버를 직접 띄우지 않고도, 보안이 강화된 MCP 도구를 엔터프라이즈 환경에서 활용 가능


In [17]:
import os
from azure.identity.aio import AzureCliCredential
from agent_framework.azure import AzureAIAgentClient
from agent_framework import HostedMCPTool, ChatMessage
from agent_framework import ChatAgent
from typing import Any


async def handle_approvals_with_thread(query: str, agent, thread):
    """Handle user approval requests for function calls."""
    result = await agent.run(query, thread=thread, store=True)
    
    while len(result.user_input_requests) > 0:
        new_input: list[Any] = []
        for user_input_needed in result.user_input_requests:
            print(
                f"\n⚠️ Function call approval needed: {user_input_needed.function_call.name}"
                f"\n   Arguments: {user_input_needed.function_call.arguments}"
            )
            # Visual feedback with dots for 1 second
            for _ in range(5):
                print(".", end="", flush=True)
                await asyncio.sleep(0.33)
            print("user approval simulated. yes! \n")
            # Auto-approve for demo purposes (in production, you'd want user input)
            user_approval = "y"  # or use: input("Approve? (y/n): ")
            new_input.append(
                ChatMessage(
                    role="user",
                    contents=[user_input_needed.create_response(user_approval.lower() == "y")],
                )
            )
        result = await agent.run(new_input, thread=thread, store=True)
    
    return result


async def hosted_mcp_demo():
    """
    Demonstrates using MCP servers hosted in Azure AI Project for enterprise-ready tool integration.
    
    This example shows how to connect to a hosted MCP server with proper approval workflow.
    Based on: https://github.com/microsoft/agent-framework/tree/main/python/samples
    """
    
    try:

        # Create Azure AI Agent client with credential
        credential = AzureCliCredential()
        
        print("=== Azure AI Agent with Hosted MCP Server ===")
        print("This agent uses hosted MCP tools to access Microsoft documentation.\n")
        
        async with AzureAIAgentClient(
            async_credential=credential,
            project_endpoint=azure_project_endpoint,
            model_deployment_name=azure_ai_deployment,
            ) as chat_client:
            
            # Enable Azure AI observability (optional but recommended)
            #await chat_client.setup_azure_ai_observability()
            
            # Create agent with hosted MCP tools using create_agent method
            agent = chat_client.create_agent(
                name="Hosted MCP Assistant(AzureAIAgentClient)",
                instructions="You are a helpful assistant that can help with Microsoft documentation questions using MCP tools.",
                tools=HostedMCPTool(
                    name="Microsoft Learn MCP",
                    url="https://learn.microsoft.com/api/mcp",
                )
            )
            
            # Create a new thread for conversation
            thread = agent.get_new_thread()
            
            # Test questions using MCP tools
            USER_INPUTS = [
                "How to create an Azure storage account using az cli?",
                "What is Microsoft Agent Framework?"
            ]
            
            # Simulate conversation with approval workflow
            for user_input in USER_INPUTS:
                print(f"User: {user_input}")
                print("Agent: ", end="", flush=True)
                
                # Handle query with approval workflow
                result = await handle_approvals_with_thread(user_input, agent, thread)
                print(f"{result.text}")
                print("\n" + "=" * 50 + "\n")
            
            print("Hosted MCP tool conversation completed!")
    
    except Exception as e:
        print(f"Error occurred: {e}")
        import traceback

    finally:
        # Cleanup: Close Azure AI Agent client if it was created
        try:
            await chat_client.close()    
        except Exception as cleanup_error:
            print(f"Cleanup warning: {cleanup_error}")

await hosted_mcp_demo()

=== Azure AI Agent with Hosted MCP Server ===
This agent uses hosted MCP tools to access Microsoft documentation.

User: How to create an Azure storage account using az cli?
Agent: 
⚠️ Function call approval needed: microsoft_docs_search
   Arguments: {"query":"create Azure storage account using az cli","question":"How to create an Azure storage account using az cli?"}
.
⚠️ Function call approval needed: microsoft_docs_search
   Arguments: {"query":"create Azure storage account using az cli","question":"How to create an Azure storage account using az cli?"}
.........user approval simulated. yes! 

user approval simulated. yes! 

To create an Azure storage account using the Azure CLI, you can use the command `az storage account create`. Here is an example command to create a standard general-purpose v2 storage account with read-access geo-redundant storage:

```azurecli
az storage account create \
  --name <account-name> \
  --resource-group storage-resource-group \
  --location eastus 

[2025-12-03 11:38:29 - /anaconda/envs/venv_agentlab/lib/python3.12/site-packages/agent_framework_azure_ai/_chat_client.py:677 - ERROR] Error processing stream: Sorry, something went wrong.


Error occurred: Sorry, something went wrong.


## 📊 Case 4 비교: 엔터프라이즈 도구



| 항목 | AI Search (4.1) | Bing Grounding (4.2) | Local MCP (4.3) | Hosted MCP (4.4) |
|------|-----------------|----------------------|-----------------|------------------|
| **도구 타입** | `HostedFileSearchTool` | `HostedWebSearchTool` | `MCPClient` | `HostedMCPTool` |
| **클라이언트 타입** | `AzureAIAgentClient` | `AzureAIAgentClient` | `AzureOpenAIChatClient` | `AzureAIAgentClient` |
| **데이터 소스** | Azure AI Search 벡터 스토어 | Bing Search API | 로컬 MCP 서버 프로세스 | Azure에 호스팅된 MCP 서버 |
| **설정 난이도** | 중간 (벡터 스토어 필요) | 낮음 (Bing 연결 필요) | 중간 (로컬 서버 필요) | 낮음 (Azure 연결 필요) |
| **주요 사용 사례** | 내부 문서 검색, 지식 베이스 | 실시간 웹 정보 | 로컬 커스텀 도구, 파일 시스템 | 엔터프라이즈급 커스텀 도구 |
| **인증 방식** | `DefaultAzureCredential` | `DefaultAzureCredential` | `AzureCliCredential` | `DefaultAzureCredential` |
| **그라운딩** | 인덱스 문서 기반 인용 | 웹 검색 결과 인용 및 출처 | N/A (커스텀 도구 출력) | MCP 서버 구현에 따라 다름 |
| **확장성** | 높음 (Azure 관리형) | 높음 (Azure 관리형) | 낮음 (로컬 프로세스) | 높음 (Azure 관리형) |
| **커스터마이징** | 인덱스 데이터 범위 내 | 웹 검색 범위 내 | 매우 높음 (커스텀 MCP 도구) | 매우 높음 (커스텀 MCP 도구) |
| **비용** | AI Search + AI Project | Bing API + AI Project | Azure OpenAI만 사용 | MCP 호스팅 + AI Project |



**요약 정리:**

- **AI Search (4.1)**: 시맨틱 검색 기반으로, 조직 내부 문서·지식 베이스 검색에 최적

- **Bing Grounding (4.2)**: 최신 웹 정보와 출처 인용이 필요한 경우 적합

- **Local MCP (4.3)**: 개발/테스트 환경에서 로컬 커스텀 도구를 빠르게 붙여보고 싶을 때 유용

- **Hosted MCP (4.4)**: 보안·확장성이 중요한 프로덕션 환경에서 커스텀 도구를 운영할 때 권장
